<a href="https://colab.research.google.com/github/petermr/semanticClimate/blob/main/abbreviation/Codes/climate_abbreviation_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/petermr/pyami.git

Cloning into 'pyami'...
remote: Enumerating objects: 5767, done.
remote: Counting objects: 100% (259/259), done.
remote: Compressing objects: 100% (182/182), done.
remote: Total 5767 (delta 147), reused 162 (delta 71), pack-reused 5508
Receiving objects: 100% (5767/5767), 103.87 MiB | 24.44 MiB/s, done.
Resolving deltas: 100% (2451/2451), done.


In [2]:
!pip install abbreviations
!pip install scispacy==0.3.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 10.4 MB 22.6 MB/s 
     |████████████████████████████████| 71 kB 9.4 MB/s 
     |████████████████████████████████| 13.5 MB 50.6 MB/s 
     |████████████████████████████████| 188 kB 98.8 MB/s 
     |████████████████████████████████| 1.0 MB 65.8 MB/s 
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [3]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [4]:
import spacy
import scispacy
from bs4 import BeautifulSoup
from abbreviations import schwartz_hearst
from scispacy.abbreviation import AbbreviationDetector

import os
for i in range(1,30):
  path = './'
  head_folder = path + f'Pages_climate_report/page_{i}' +'/'
  if not os.path.exists(head_folder):
    os.makedirs(head_folder)
  print(f'{i}---------------')
  with open(f'/content/pyami/temp/html/page.{i}.html', 'r') as f:
      html = f.read()
      soup = BeautifulSoup(html, features="html.parser")

      # kill all script and style elements
      for script in soup(["script", "style"]):
          script.extract()    # rip it out

      # get text
      text = soup.get_text()

      # break into lines and remove leading and trailing space on each
      lines = (line.strip() for line in text.splitlines())
      # break multi-headlines into a line each
      chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
      # drop blank lines
      #text_write = '\n'.join(chunk for chunk in chunks if chunk)
      text = '\n'.join(chunk for chunk in chunks if chunk)

      #print(text)
      pairs = schwartz_hearst.extract_abbreviation_definition_pairs(doc_text= text)
      #print('pairs',pairs)
      
      TEXT_ = f'page_{i}_text.txt'
      DICTIONARY_SH = f'page_{i}_dictionary_SH.txt'
      DICTIONARY_Spacy= f'page_{i}_dictionary_Spacy.txt'
      with open(head_folder + TEXT_, 'w') as file:
          file.write(text)
      
      with open(head_folder + DICTIONARY_SH, 'w') as file:
          for key, value in pairs.items():
            #print(key, value )
            file.write('{'+str(key)+' : '+ str(value)+"}")
            file.write('\n')
      

      nlp = spacy.load("en_core_web_sm")

      abbreviation_pipe = AbbreviationDetector(nlp)
      nlp.add_pipe(abbreviation_pipe)
      doc = nlp(text)
      altered_tok = [tok.text for tok in doc]
      with open(head_folder + DICTIONARY_Spacy, 'w') as file:
          for abrv in doc._.abbreviations:
            
            #print(abrv ,abrv._.long_form)
            #print(key, value )
            file.write('{'+str(abrv)+' : '+ str(abrv._.long_form)+"}")  
            file.write('\n')    

hello
EVs electric vehicles
13---------------
14---------------
hello
LCOE levelised costs of energy

hello
LCOE levelised costs of energy

hello
LCOE levelised costs of energy

hello
LCOE levelised costs of energy

15---------------
hello
ESG environmental, social and
governance
hello
NDCs nationally
determined contributions
hello
NDCs nationally
determined contributions
hello
NDCs nationally
determined contributions
hello
NDCs nationally
determined contributions
16---------------
17---------------
18---------------
19---------------
20---------------
21---------------
22---------------
23---------------
24---------------
25---------------
CDR carbon dioxide removal
hello
CDR carbon dioxide removal
26---------------
27---------------
28---------------
29---------------


#using scispacy example

In [9]:
# #!pip install scispacy==0.3.0
# import spacy
# import scispacy
# print(scispacy.__version__)
# from scispacy.abbreviation import AbbreviationDetector
# #from scispacy.abbreviation import AbbreviationDetector

# nlp = spacy.load("en_core_web_sm")

# abbreviation_pipe = AbbreviationDetector(nlp)
# nlp.add_pipe(abbreviation_pipe)

# text = "StackOverflow (SO) is a question and answer site for professional and enthusiast programmers. SO rocks!"


# doc = nlp(text)
# altered_tok = [tok.text for tok in doc]
# for abrv in doc._.abbreviations:
#   print(abrv ,abrv._.long_form)
    
# #replace_acronyms(text)

0.3.0


/usr/local/lib/python3.7/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.7). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


SO StackOverflow
SO StackOverflow
